# Data exploration
We gaan kijken naar outliers en ontbrekende data voor de podcast database. Aan de hand van dit document laat ik zien hoe ik tot de filtering / data manipulatie in de main notebook ben gekomen.

In [1]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine


db_connection_str = 'mysql+pymysql://root:@localhost/podcast_feeds'
db_connection = create_engine(db_connection_str)
print("Connection established")

Connection established


De kolommen van deze database die nodig zijn om mijn vragen mee te beantwoorden zijn: oldestItemPubdate, newestItemPubdate, episodeCount, language en host

### oldestItemPubdate 
Zijn er null values?


In [2]:
query = '''
    SELECT SUM(CASE WHEN `oldestItemPubdate` is Null THEN 1 ELSE 0 END) 
        AS Nullvalues_oldestItem
    FROM `podcasts`
    WHERE from_unixtime(oldestItemPubdate, '%%Y') BETWEEN 2020 AND 2020;
'''
pd.read_sql(query, con=db_connection)

,Nullvalues_oldestItem
0,0.0


Welke WHERE query kan gebruikt worden om alle podcasts die in 2020 gemaakt zijn toe te voegen?

In [4]:
query = '''
    SELECT MIN(oldestItemPubdate)
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020; 
'''
podcasts_df = pd.read_sql(query, con=db_connection)
podcasts_df["converted"] = pd.to_datetime(podcasts_df["MIN(oldestItemPubdate)"], unit="s")
print(podcasts_df)


query = '''
    SELECT MAX(oldestItemPubdate)
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020;
'''
podcasts_df = pd.read_sql(query, con=db_connection)
podcasts_df["converted"] = pd.to_datetime(podcasts_df["MAX(oldestItemPubdate)"], unit="s")
print(podcasts_df)

   MIN(oldestItemPubdate)           converted
0              1577833200 2019-12-31 23:00:00
   MAX(oldestItemPubdate)           converted
0              1609455589 2020-12-31 22:59:49


Goed genoeg!

### newestItemPubdate
Zijn er null values?


In [5]:
query = '''
    SELECT SUM(CASE WHEN `oldestItemPubdate` is Null THEN 1 ELSE 0 END) 
        AS Nullvalues_oldestItem
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020;
'''
pd.read_sql(query, con=db_connection)

,Nullvalues_oldestItem
0,0.0


Zijn er podcasts waarbij de timedelta newest - oldest negatief is, oftewel, waar de datum voor newest ouder is dan van oldest?

In [6]:
query = '''
    SELECT SUM(CASE WHEN(newestItemPubdate - oldestItemPubdate < 0) THEN 1 END)
            AS timedelta_negatief,
        SUM(CASE WHEN(newestItemPubdate - oldestItemPubdate >= 0) THEN 1 END)
            AS timedelta_positief
            
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020;
'''
pd.read_sql(query, con=db_connection)

,timedelta_negatief,timedelta_positief
0,None,1682168.0


Alle rijen in kolommen oldestItemPubdate en newestItemPubdate zijn valide.

### episodeCount
Zijn er podcasts met een episodecount < 0?

In [7]:
query = '''
    SELECT SUM(CASE WHEN `episodeCount` = 0 THEN 1 END)
            AS EpisodeCount_0,
        SUM(CASE WHEN `episodeCount` = 1 THEN 1 END)
            AS EpisodeCount_1,
        SUM(CASE WHEN `episodeCount` > 1 THEN 1 END)
            AS EpisodeCount_rest
    
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020;
'''
pd.read_sql(query, con=db_connection)

,EpisodeCount_0,EpisodeCount_1,EpisodeCount_rest
0,5562.0,697482.0,979124.0


Er zijn dus 5.6k podcasts die geen episodeCount hebben, en dus zowieso weggehaald moeten worden via <code>WHERE `episodeCount` > 0</code>. Het is ook opvallend dat bijna 700k podcasten maar 1 aflevering hebben.

De SQL query is dus: <code>
query = '''
    SELECT *
    FROM `podcasts`
    WHERE from_unixtime(`oldestItemPubdate`, '%%Y') BETWEEN 2020 AND 2020
        AND `episodeCount` > 0 
'''
</code>

### language
Zijn er podcasts zonder valide value in kolom language? 

In [8]:
query = '''
    SELECT `language`, COUNT(*) AS `count`
    FROM `podcasts` 
    WHERE from_unixtime(oldestItemPubdate, '%%Y') BETWEEN 2020 AND 2020
        AND `episodeCount` > 0 
    GROUP BY `language`
'''
podcasts_df = pd.read_sql(query, con=db_connection)
podcasts_df.head()

,language,count
0,,694
1,en,2
2,#,5
3,aa,54
4,ab,57


De rijen die er uitgehaald moeten worden zijn <code>podcasts_df["language"] = "" </code>en <code>podcasts_df["language"] = #</code>. Verder moet de data getransformeerd worden, zodat de categorieën kleiner worden.

### host
Zijn er podcasts zonder value in de host kolom?

In [9]:
query = '''
    SELECT `host`, COUNT(*) AS `count`
    FROM `podcasts` 
    WHERE from_unixtime(oldestItemPubdate, '%%Y') BETWEEN 2020 AND 2020
        AND `episodeCount` > 0 
    GROUP BY `host`
'''
podcasts_df = pd.read_sql(query, con=db_connection)
podcasts_df.head()

,host,count
0,000webhostapp.com,1
1,1-page.biz,1
2,100-sekunden.ch,1
3,1000-cupsoftea.com,1
4,1001moviesandbeyond.com,1


Nee, iedere podcast heeft een value. Maar er zijn heel veel hosts met maar 1 entry.